In [1]:
import pandas as pd
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [2]:
passing = 'NFL_STATISCS/Career_Stats_Passing.csv'
reciving = 'NFL_STATISCS/Career_Stats_Receiving.csv'
rushing = 'NFL_STATISCS/Career_Stats_Rushing.csv'
qb = 'NFL_STATISCS/Game_Logs_Quarterback.csv'
basic = 'NFL_STATISCS/Basic_Stats.csv'
runnungBack = 'NFL_STATISCS/Game_Logs_Runningback.csv'
receiver = 'NFL_STATISCS/Game_Logs_Wide_Receiver_and_Tight_End.csv'

In [3]:
#passing_df = pd.read_csv(file_passing)
#passing_df.loc[passing_df['Player Id']=='billdemory/2512778',:]

In [4]:
# Create all the DataFrames
qb_df = pd.read_csv(qb)             # Len = 40,247
runnungBack_df = pd.read_csv(runnungBack) #Len = 67,661   nan, 'RB', 'FB'    #11,626 That Contain Position 
receiver_df = pd.read_csv(receiver)       #Len 112,632    nan, 'WR', 'TE',   # 24,786 That Contain Position 


passing_df = pd.read_csv(passing) # Len = 8,525 First Year = 1924
rushing_df = pd.read_csv(rushing) # Len = 17,507 First Year = 1924
reciving_df = pd.read_csv(reciving) #Len = 18,128
basic_df = pd.read_csv(basic)       #Len = 17,172



In [5]:
#Cleaning the QB Data Frame

qb_df['Position'] = qb_df['Position'].fillna('QB')

qb_clean_df = qb_df.loc[(qb_df['Season']=='Regular Season') | (qb_df['Season']=='Postseason'),:].replace('--',0)
qb_clean_df

qb_clean_df = qb_clean_df.rename(columns = {'Player Id': 'Player_id',
                                           'Games Played':'Games_Played',
                                          'Games Started':'Games_Started',
                                            'Passes Completed' :'Passes_Completed',
                                            'Passes Attempted':'Passes_Attempted'})


columns = ['Player_id', 'Name', 'Position','Year', 'Season', 'Games_Played', 
           'Games_Started', 'Passes_Completed', 'Passes_Attempted']
qb_clean_df = qb_clean_df.reset_index(drop= True)
qb_df_sql = qb_clean_df.loc[:,columns]

qb_df_sql

,Player_id,Name,Position,Year,Season,Games_Played,Games_Started,Passes_Completed,Passes_Attempted
0,billdemory/2512778,"Demory, Bill",QB,1974,Regular Season,0,0,0,0
1,billdemory/2512778,"Demory, Bill",QB,1974,Regular Season,0,0,0,0
2,billdemory/2512778,"Demory, Bill",QB,1974,Regular Season,0,0,0,0
3,billdemory/2512778,"Demory, Bill",QB,1974,Regular Season,0,0,0,0
4,billdemory/2512778,"Demory, Bill",QB,1974,Regular Season,0,0,0,0
...,...,...,...,...,...,...,...,...,...
36181,steveramsey/2523725,"Ramsey, Steve",QB,1971,Regular Season,1,0,8,16
36182,steveramsey/2523725,"Ramsey, Steve",QB,1971,Regular Season,1,0,10,25
36183,steveramsey/2523725,"Ramsey, Steve",QB,1971,Regular Season,1,0,16,38
36184,steveramsey/2523725,"Ramsey, Steve",QB,1970,Regular Season,0,0,0,0


In [6]:
#DATA FRAME THAT contains unique players

qb_id = qb_df_sql['Player_id'].unique()
qb_name = qb_df_sql['Name'].unique()

#qb_id_name = pd.DataFrame(qb_id,qb_name, columns  = ['Player_id','Player_id'])
qb_id_name = pd.DataFrame({'Player_id': qb_id, 'Name' : qb_name})#, columns  = ['Player_id','Player_id'])
qb_id_name


,Player_id,Name
0,billdemory/2512778,"Demory, Bill"
1,tomsavage/2543640,"Savage, Tom"
2,tommymaddox/2501842,"Maddox, Tommy"
3,jimbobtaylor/2527093,"Taylor, Jim Bob"
4,jaybarker/2499535,"Barker, Jay"
...,...,...
462,alpastrana/2522827,"Pastrana, Al"
463,aaronrodgers/2506363,"Rodgers, Aaron"
464,chrisweinke/2504686,"Weinke, Chris"
465,brockosweiler/2533436,"Osweiler, Brock"


In [7]:
# Cleaning RB Data Frame UNIQUE QB AND NANMES

rb_df = runnungBack_df.loc[(runnungBack_df['Position']== 'RB') | (runnungBack_df['Position']== 'FB'),:]
rb_df
runnungBack_clean_df = rb_df.loc[(rb_df['Season']=='Regular Season') | (rb_df['Season']=='Postseason'),:].reset_index(drop=True)
runnungBack_clean_df

,Player Id,Name,Position,Year,Season,Week,Game Date,Home or Away,Opponent,Outcome,...,Yards Per Carry,Longest Rushing Run,Rushing TDs,Receptions,Receiving Yards,Yards Per Reception,Longest Reception,Receiving TDs,Fumbles,Fumbles Lost
0,kenyandrake/2555230,"Drake, Kenyan",RB,2016,Regular Season,1,09/11,Away,SEA,L,...,--,--,--,--,--,--,--,--,--,--
1,kenyandrake/2555230,"Drake, Kenyan",RB,2016,Regular Season,2,09/18,Away,NE,L,...,6.0,7T,1,2,6,3.0,4,0,--,--
2,kenyandrake/2555230,"Drake, Kenyan",RB,2016,Regular Season,3,09/25,Home,CLE,W,...,4.1,11,0,2,11,5.5,7,0,--,--
3,kenyandrake/2555230,"Drake, Kenyan",RB,2016,Regular Season,4,09/29,Away,CIN,L,...,7.5,12,0,--,--,--,--,--,--,--
4,kenyandrake/2555230,"Drake, Kenyan",RB,2016,Regular Season,5,10/09,Home,TEN,L,...,2.0,6,0,2,16,8.0,9,0,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8950,derekwatt/2555220,"Watt, Derek",FB,2016,Regular Season,13,12/04,Home,TB,L,...,2.0,2,0,--,--,--,--,--,--,--
8951,derekwatt/2555220,"Watt, Derek",FB,2016,Regular Season,14,12/11,Away,CAR,L,...,--,--,--,--,--,--,--,--,--,--
8952,derekwatt/2555220,"Watt, Derek",FB,2016,Regular Season,15,12/18,Home,OAK,L,...,--,--,--,--,--,--,--,--,--,--
8953,derekwatt/2555220,"Watt, Derek",FB,2016,Regular Season,16,12/24,Away,CLE,L,...,--,--,--,1,16,16.0,16,0,--,--


In [8]:
#receiver_df = pd.read_csv(receiver)       #Len 112,632    nan, 'WR', 'TE',   # 24,786 That Contain Position 


rec_df = receiver_df.loc[(receiver_df['Position']== 'WR') | (receiver_df['Position']== 'TE'),:]
rec_df
receiver_clean_df = rec_df.loc[(rec_df['Season']=='Regular Season') | (rec_df['Season']=='Postseason'),:].reset_index(drop=True)
receiver_clean_df


,Player Id,Name,Position,Year,Season,Week,Game Date,Home or Away,Opponent,Outcome,...,Yards Per Reception,Longest Reception,Receiving TDs,Rushing Attempts,Rushing Yards,Yards Per Carry,Longest Rushing Run,Rushing TDs,Fumbles,Fumbles Lost
0,kevinnorwood/2543689,"Norwood, Kevin",WR,2015,Regular Season,1,09/13,Away,JAC,W,...,--,--,--,--,--,--,--,--,--,--
1,kevinnorwood/2543689,"Norwood, Kevin",WR,2015,Regular Season,2,09/20,Home,HOU,W,...,--,--,0,--,--,--,--,--,--,--
2,kevinnorwood/2543689,"Norwood, Kevin",WR,2015,Regular Season,3,09/27,Home,NO,W,...,--,--,--,--,--,--,--,--,--,--
3,kevinnorwood/2543689,"Norwood, Kevin",WR,2015,Regular Season,4,10/04,Away,TB,W,...,--,--,--,--,--,--,--,--,--,--
4,kevinnorwood/2543689,"Norwood, Kevin",WR,2015,Regular Season,6,10/18,Away,SEA,W,...,--,--,--,--,--,--,--,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19069,torreysmith/2495459,"Smith, Torrey",WR,2011,Regular Season,15,12/18,Away,SD,L,...,12.8,36T,1,--,--,--,--,--,1,0
19070,torreysmith/2495459,"Smith, Torrey",WR,2011,Regular Season,16,12/24,Home,CLE,W,...,19.0,29,0,--,--,--,--,--,--,--
19071,torreysmith/2495459,"Smith, Torrey",WR,2011,Regular Season,17,01/01,Away,CIN,W,...,6.6,13,0,--,--,--,--,--,--,--
19072,torreysmith/2495459,"Smith, Torrey",WR,2011,Postseason,19,01/15,Home,HOU,W,...,9.0,9,0,--,--,--,--,--,--,--


In [9]:
conn_string = 'postgres:postgres@localhost:5432/NFL_Statistics'
engine = create_engine(f'postgresql://{conn_string}')
Base = declarative_base()

In [10]:
class QB_NAMES(Base):
    __tablename__ = 'QB_NAMES'
    Player_id = Column(String,primary_key = True)
    Name = Column(String)

class RB(Base):
    __tablename__ = 'RB'
    Player_id = Column(String, primary_key = True)
    name = Column(String)
    Position = Column(String)
    Year = Column(Integer)
    Season = Column(String)
    Games_Played = Column(Integer)
    Games_Started = Column(Integer)
    Rushing_Yards = Column(Integer)
    Rushing_Attempts = Column(Integer)

In [11]:
Base.metadata.tables

immutabledict({'QB_NAMES': Table('QB_NAMES', MetaData(bind=None), Column('Player_id', String(), table=<QB_NAMES>, primary_key=True, nullable=False), Column('Name', String(), table=<QB_NAMES>), schema=None)})

In [12]:
# This is where we create our tables in the database
Base.metadata.create_all(engine)

In [13]:
from sqlalchemy.orm import Session
session = Session(engine)

In [18]:
pd.read_sql('SELECT * FROM "QB_NAMES"', engine)


,Player_id,Name


In [23]:
session.new

IdentitySet([])

In [25]:
#QB_NAMES
qb_id_name.to_sql(name="QB_NAMES",con=engine,if_exists='replace', index=False)

In [21]:
qb_id_name

,Player_id,Name
0,billdemory/2512778,"Demory, Bill"
1,tomsavage/2543640,"Savage, Tom"
2,tommymaddox/2501842,"Maddox, Tommy"
3,jimbobtaylor/2527093,"Taylor, Jim Bob"
4,jaybarker/2499535,"Barker, Jay"
...,...,...
462,alpastrana/2522827,"Pastrana, Al"
463,aaronrodgers/2506363,"Rodgers, Aaron"
464,chrisweinke/2504686,"Weinke, Chris"
465,brockosweiler/2533436,"Osweiler, Brock"


In [20]:

session.close()